# Load Environment

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
from IPython.display import set_matplotlib_formats
%matplotlib inline
%config InlineBackend.figure_format='svg'
import seaborn
seaborn.reset_orig()
from matplotlib import rcParams
rcParams['figure.figsize'] = 10, 4

import numpy as np
from optalg.opt_solver import *
import scipy.io # to read matlab files

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
from mppfnet.mp_network import MPNetwork
from mppfnet.mp_problem import MPProblem
import mppfnet
import gridopt
import pfnet

# Load Network

In [3]:
# Construct multi-period network and load topology
mp = MPNetwork(timesteps=24)
mp.load('./data/case32.art')

print(mp.base_power)

1.0


In [4]:
# Load EEX prices
prices = scipy.io.loadmat('./data/eex_intraday_2010_3600s.mat')['eex_intraday_2010_3600s']
eex_prices = np.array([p[0,0] for p in prices[:, 5][1:]])
eex_prices.shape
# Set prices to EEX prices
mp.set_prices(eex_prices)

# Load Profiles
mp.load_load_profile_from_csv("./data/Electricity_Profile.csv")

## Generate Vargen Profiles

In [5]:
mp.generate_solar_profiles()

# Construct Optimization Problem

In [134]:
# bus voltage angles
mp.set_flags(pfnet.OBJ_BUS, pfnet.FLAG_VARS, pfnet.BUS_PROP_NOT_SLACK, pfnet.BUS_VAR_VANG)

# Batteries must be variables
mp.set_flags(pfnet.OBJ_BAT, pfnet.FLAG_VARS, pfnet.BAT_PROP_ANY, pfnet.BAT_VAR_P)
mp.set_flags(pfnet.OBJ_BAT, pfnet.FLAG_VARS, pfnet.BAT_PROP_ANY, pfnet.BAT_VAR_E)

# Mark Gen, Load and Vargen as variables
#mp.set_flags(pfnet.OBJ_LOAD, pfnet.FLAG_VARS, pfnet.LOAD_PROP_ANY, pfnet.LOAD_VAR_P)
#mp.set_flags(pfnet.OBJ_GEN, pfnet.FLAG_VARS,pfnet.GEN_PROP_ANY, pfnet.GEN_VAR_P)
#mp.set_flags(pfnet.OBJ_VARGEN, pfnet.FLAG_VARS, pfnet.VARGEN_PROP_ANY, pfnet.VARGEN_VAR_P)

# slack gens active powers
mp.set_flags(pfnet.OBJ_GEN, pfnet.FLAG_VARS, pfnet.GEN_PROP_SLACK,pfnet.GEN_VAR_P)

# gens active powers
mp.set_flags(pfnet.OBJ_GEN,pfnet.FLAG_BOUNDED,pfnet.GEN_PROP_ANY,pfnet.GEN_VAR_P)

# Battery Bounds
mp.set_flags(pfnet.OBJ_BAT,pfnet.FLAG_BOUNDED,pfnet.BAT_PROP_ANY,pfnet.BAT_VAR_P)
mp.set_flags(pfnet.OBJ_BAT,pfnet.FLAG_BOUNDED,pfnet.BAT_PROP_ANY,pfnet.BAT_VAR_E)

# Fix Load
#mp.set_flags(pfnet.OBJ_LOAD, pfnet.FLAG_FIXED, pfnet.LOAD_PROP_ANY, pfnet.LOAD_VAR_P)

# Fix Vargen
#mp.set_flags(pfnet.OBJ_VARGEN, pfnet.FLAG_FIXED, pfnet.VARGEN_PROP_ANY, pfnet.VARGEN_VAR_P)

# Construct Problem
p = MPProblem(mp)
p.add_function(pfnet.FUNC_TYPE_NETCON_COST, 1.0)
p.add_constraint(pfnet.CONSTR_TYPE_LBOUND)  # generator bounds
p.add_constraint(pfnet.CONSTR_TYPE_DCPF)  # power flow
p.add_constraint(pfnet.CONSTR_TYPE_FIX) # fix load and vargen
p.add_constraint(mppfnet.CONSTR_TYPE_BAT_DYN) # battery dynamic constraints

p.analyze()
x = p.get_init_point()
p.eval(x)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  6.,  3.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  6.,  3.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        6.,  3.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.,  3.,
        2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.,  3.,  2

In [7]:
# Helper variables
nx = mp.networks[0].num_vars

# Show information about network
mp.show_components()

Network Components
-----------------------------
timesteps:           : 24
------- per timestep --------
buses:               : 31
   slack:            : 1
   reg by gen:       : 1
   reg by tran:      : 1
   reg by shunt:     : 0
shunts:              : 0
   fixed:            : 0
   switched v:       : 0
branches:            : 30
   lines:            : 29
   fixed trans:      : 0
   phase shifters    : 0
   tap changers v    : 1
   tap changers Q    : 0
generators:          : 1
   slack:            : 1
   reg:              : 1
   P adjust          : 1
loads:               : 19
   P adjust:         : 0
vargens:             : 2
batteries:           : 3


# Construct optimization problem for linear solver

In [8]:
from optalg.opt_solver.lp_cvxopt import OptSolverLP_CVXOPT
from optalg.opt_solver.linear_problem import LinProblem
from scipy.sparse import triu

Hx = p.Hphi + p.Hphi.T - triu(p.Hphi)
gx = p.gphi - Hx*x

g = gx / mp.base_power
        
H = Hx / mp.base_power
A = p.A
b = p.b
l = p.l
u = p.u

problem = QuadProblem(H, g, A, b, l, u, x=p.x)
solver = OptSolverIQP()
solver.set_parameters({'quiet': False, 'tol': 1e-4})
problem.show()

## Solve Optimization Problem

In [ ]:
solver.solve(problem)
print(solver.get_status())
problem.x

## Attempt to solve the problem with ECOS

In [ ]:
#import ecos
#import scipy
#dims = dict(l=9744, q=[])
#G = scipy.sparse.coo_matrix(([], [[], []]), shape=(9744,9744))
#h = zeros((9744,))
#solution = ecos.solve(g,G,h,dims, A=p.A,b=p.b,)

## Store Solver Result

In [ ]:
mp.set_var_values(solver.get_primal_variables())

# Output

## Battery SOC

In [ ]:
%aimport pfvis

pfvis.plot_battery_soc(mp)


## Battery Power

In [ ]:
%aimport pfvis

pfvis.plot_battery_power(mp)


## Energy Price

In [ ]:
pfvis.plot_energy_price(mp)

## Load Power

In [ ]:
pfvis.plot_load_power(mp)

## Vargen Injection

In [ ]:
pfvis.plot_vargen_injection(mp)

## Plot Network Graph

In [ ]:
#g = pfnet.Graph(mp.networks[0])
#for bus in mp.networks[0].buses:
#    g.set_node_property(bus,b"label",str(bus.number).encode('UTF-8'))
#g.set_layout()    
#g.view(inline=True)

# Distributed Approach

In [135]:
prob = p.problems[0]
prob.analyze()
constraint = prob.constraints[1]
constraint.A.shape

(31, 37)

In [132]:
net = prob.network
net.var_generators[0].index_P

0

In [139]:
# Primal Dual Gradient
iteration = 1

nx = constraint.A.shape[1]
nc = constraint.A.shape[0]
A = constraint.A
b = constraint.b
g = zeros((nx,))

g[36] = 1

print(g)

H = zeros((nx, nx))
l = - 1e2 * ones((nx,))
u = 1e2 * ones((nx,))

print(A.shape)
print(b.shape)
print(g.shape)
print(H.shape)
print(l.shape)
print(u.shape)
x = zeros((nx,))

A.shape
problem = QuadProblem(H, g, A, b, l, u, x=x)
solver = OptSolverIQP()
solver.set_parameters({'quiet': False, 'tol': 1e-4})
solver.solve(problem)

ld = zeros((nc, ))
x = zeros((nx,))

print(solver.get_dual_variables()[0])

print("\n\n")
print("Solver: Primal-Dual Gradient")
print("----------------------------")
print("it    obj           alpha         lmin           lmax         dL/dx         dL/dl")
 
for iteration in range(1, 2000):
    alpha = 1 / (iteration + 1)

    lx = (g + x + A.T * ld) # add H here?
    ll = (A * x - b)
    #lx = lx / norm(lx)
    #ll = ll/ norm(ll)
    xprev = x.copy()
    x = np.minimum(np.maximum(x - alpha * lx, l), u)
    ld += alpha * ll
    print("{0}     {1:.2E}      {2:.2E}      {3:.2E}      {4:.2E}     {5:.2E}      {6:.2E}     {7:.2E}".format(iteration, g.dot(x),  alpha, min(ld), max(ld), norm(lx), norm(ll), norm(x - xprev)))

    



[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.]
(31, 37)
(31,)
(37,)
(37, 37)
(37,)
(37,)

Solver: IQP
-----------
iter    phi      fmax      gmax       cu       cl       s    
 0  0.00e+00  1.00e+00  1.34e+02  1.0e+00  1.0e+00  0.0e+00 
 1  -4.00e+02 9.84e-01  1.32e+02  9.9e-01  9.9e-01  1.6e-02 
 2  -4.48e+02 9.72e-01  1.31e+02  9.8e-01  9.8e-01  1.2e-02 
 3  -4.69e+02 9.43e-01  1.27e+02  9.5e-01  9.5e-01  3.0e-02 
 4  -4.68e+02 7.02e-01  9.43e+01  7.3e-01  7.3e-01  2.6e-01 
 5  -4.70e+02 4.25e-01  5.71e+01  4.8e-01  4.8e-01  3.9e-01 
 6  -4.69e+02 7.74e-02  1.39e+01  1.4e-01  1.7e-01  1.0e+00 
 7  -4.70e+02 1.32e-02  6.80e-01  1.0e-01  1.0e-01  1.0e+00 

iter    phi      fmax      gmax       cu       cl       s    
 7  -4.70e+02 9.36e-02  1.80e+01  1.0e-01  1.0e-01  1.0e+00 
 8  -4.70e+02 5.02e-03  7.34e-01  1.3e-02  1.5e-02  1.0e+00 

iter    phi      fmax      gmax       cu     

In [ ]:
[load.P * 1e6 for load in mp.networks[0].loads]

In [ ]:
p.problems[0].A

In [ ]:
from optalg.opt_solver.decoupled_problem import DecoupledProblem
from optalg.opt_solver import OptSolverDPDG

dp = DecoupledProblem( g, a, b, l, u, [])

dp.A.tocsr()[0,1]

scipy.sparse.hstack([dp.A.getcol(0 * nx + i) for i in range(4)])
solver = OptSolverDPDG()
solver.solve(dp)